In [5]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import numpy as np

## 7.4 합성곱/풀링 계층 구현하기
### 7.4.1 4차원 배열

In [6]:
x = np.random.rand(10, 1, 28, 28)
x.shape
x[0].shape
x[0,0] # 또는 x[0][0]


(10, 1, 28, 28)

(1, 28, 28)

array([[8.57576995e-01, 2.31353362e-01, 4.19302576e-01, 4.56531622e-02,
        9.79077449e-01, 9.55583965e-01, 7.59595768e-01, 4.99164764e-01,
        1.76963855e-01, 6.67956485e-01, 8.03841403e-01, 6.38671455e-01,
        2.46063093e-01, 8.64311041e-01, 7.07707995e-01, 6.39860818e-01,
        9.35229841e-01, 1.96423428e-01, 1.16399548e-01, 9.41394386e-01,
        5.44215259e-01, 1.57219071e-01, 5.13452811e-01, 9.42929228e-01,
        6.42299892e-01, 2.22457362e-02, 2.08270862e-01, 3.41825434e-01],
       [7.08179523e-01, 4.27372512e-01, 6.10696137e-01, 4.52230688e-02,
        6.06019304e-01, 4.01708516e-01, 5.39773422e-01, 1.72238700e-01,
        1.81446560e-01, 4.35271575e-01, 3.60255715e-01, 9.56438686e-01,
        3.15940748e-01, 1.26609446e-01, 4.59762280e-01, 6.75565642e-01,
        1.22513153e-01, 4.13963557e-01, 5.32693497e-01, 5.04579094e-01,
        8.18962529e-01, 6.96261754e-01, 7.13772193e-01, 5.44987212e-01,
        6.07141992e-01, 4.05057027e-01, 1.79570584e-01, 9.41948

### 7.4.3 합성곱 계층 구현하기

In [7]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7) #(데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [8]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape #필터의 개수, 채널 수, 필터의 높이, 필터의 너비
        N, C, H, W = x.shape # 데이터 개수, 채널 수, 데이터 높이, 데이터 너비
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride) # 합성곱 신경망 통과 후 높이
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride) # 합성곱 신경망 통과 후 너비

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T #필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2) # 인덱스로 축의 순서 변경 p247

        return out


In [9]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h)/ self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 최댓값 (2)
        out = np.max(col, axis=1)

        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

        return out

In [ ]:
from collections import OrderedDict
from common.layers import Relu, Affine, SoftmaxWithLoss
class SimpleConvNet:
    def __init__(self, input_dim = (1,28,28),
                 conv_param={'filter_num':30, 'filter_size':5,
                            'pad':0, 'stride':1},
                 hidden_size = 100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2)* (conv_output_size/2))

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size) #CNN 가중치
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size) # 완전연결계층 가중치
        self.parmas['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size) # 완전연결계층 가중치
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'],conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()



